In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

# Completion model

In [2]:
from langchain_openai import OpenAI

In [3]:
llmModel = OpenAI()

# Chat Completion Model

In [4]:
from langchain_openai import ChatOpenAI

chatModel = ChatOpenAI(model = "gpt-4.1-nano-2025-04-14")

## Prompts and Prompt Templates

In [5]:
# This is for Completion Model
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    "Tell me a {adjective} story about {topic}"
)

llmModelPrompt = prompt_template.format(
    adjective = "curious",
    topic = "Machine Learning"
)

response = llmModel.invoke(llmModelPrompt)
print(response)



In the 1990s, a group of researchers at IBM were experimenting with machine learning algorithms to see if they could predict stock market trends. They trained their algorithm on decades of stock market data and were surprised to find that it was able to accurately predict market fluctuations with over 90% accuracy.

Excited by their discovery, the researchers decided to put their algorithm to the test and used it to predict the stock market for the following week. To their amazement, the algorithm was wrong in almost all of its predictions.

Confused and disappointed, the researchers decided to dig deeper into their algorithm's decision-making process. They soon discovered that the algorithm had learned to make its predictions based on the state of the weather. It turned out that on days where the weather was sunny, the stock market tended to go up, and on days where the weather was cloudy, the stock market tended to go down.

While this may seem like a strange and unreliable correla

In [6]:
# Chat Completion Model

from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an {profession} expert on {topic}."),
        ("human", "Hello, Mr. {profession}, can you please answer a question?"),
        ("ai", "Sure!"),
        ("human", "{user_input}"),
    ]
)

messages = chat_template.format_messages(
    profession="Artificial Intelligence Professor",
    topic="Generative AI",
    user_input="How is Machine learning different from Deep Learning and where falls Generative AI?"
)

response = chatModel.invoke(messages)
print(response.content)


Great question! Let me clarify the distinctions among Machine Learning, Deep Learning, and Generative AI, and how they relate to each other.

**1. Machine Learning (ML):**  
- **Definition:** A subset of Artificial Intelligence (AI) where algorithms enable systems to learn from data and improve over time without being explicitly programmed for every task.  
- **Scope:** Broad; includes many techniques like decision trees, support vector machines, k-nearest neighbors, etc.  
- **Functionality:** Finds patterns in data to make predictions or decisions.  
- **Example:** Spam email classifiers, recommendation systems.

**2. Deep Learning (DL):**  
- **Definition:** A specialized subset of machine learning that uses neural networks with many layers (hence "deep") to automatically learn feature representations from raw data.  
- **Scope:** Focused on neural network architectures like Convolutional Neural Networks (CNNs), Recurrent Neural Networks (RNNs), Transformers, etc.  
- **Capabilities

# Old way

In [7]:
from langchain_core.messages import SystemMessage
from langchain_core.prompts import HumanMessagePromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=(
                "You are an Historian expert on the Kennedy family."
            )
        ),
        HumanMessagePromptTemplate.from_template("{user_input}"),
    ]
)

messages = chat_template.format_messages(
    user_input="Name the children and grandchildren of Joseph P. Kennedy?"
)

response = chatModel.invoke(messages)

In [8]:
print(response.content)

Joseph P. Kennedy Sr. and his wife, Rose Kennedy, had nine children, many of whom further contributed to the Kennedy family legacy through their own children. Here are their children and, where applicable, their grandchildren:

**Children of Joseph P. Kennedy and Rose Kennedy:**

1. **Joseph P. Kennedy Jr. (1915–1944)**  
   - No children; died in World War II.

2. **John F. Kennedy (1917–1963)**  
   - Married Jacqueline Kennedy.  
   - Children:
     - Caroline Kennedy (born 1957)
     - John F. Kennedy Jr. (1960–1999)
     - Caroline Kennedy Schlossberg (born 1957, daughter of John F. Kennedy and Jacqueline)

3. **Joseph P. Kennedy II (born 1952)**  
   - Son of Robert F. Kennedy, not Joseph P. Kennedy Sr.  
   - *Note:* Joseph P. Kennedy II is a grandson, not a son of Joseph P. Kennedy Sr.

4. **Rose Kennedy Schlossberg (born 1988)**  
   - Daughter of Caroline Kennedy, granddaughter of Joseph P. Kennedy Sr.

5. **Kathleen "Kick" Kennedy (1920–1948)**  
   - Died unmarried; no chil

## Few Shot Prompting

In [9]:
from langchain_core.prompts import FewShotChatMessagePromptTemplate

examples = [
    {"input": "hi!", "output": "¡hola!"},
    {"input": "bye!", "output": "¡adiós!"},
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an English-Spanish translator."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

# Chains

In [10]:
from langchain_core.prompts import FewShotChatMessagePromptTemplate

examples = [
    {"input": "hi!", "output": "¡hola!"},
    {"input": "bye!", "output": "¡adiós!"},
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an English-Spanish translator."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

chain = final_prompt | chatModel

response = chain.invoke({"input" : "How are you?"})
print(response.content)

¿Cómo estás?


In [11]:
# Langchain Expressions Language

## Output Parsers

In [13]:
from langchain_openai import OpenAI

llmModel = OpenAI()

In [12]:
from langchain_openai import ChatOpenAI

chatModel = ChatOpenAI(model = "gpt-4.1-nano-2025-04-14")

In [14]:
from langchain_core.prompts import PromptTemplate
from langchain.output_parsers.json import SimpleJsonOutputParser

json_prompt = PromptTemplate.from_template(
    "Return a JSON object with an `answer` key that answers the following question: {question}"
)

json_parser = SimpleJsonOutputParser()

json_chain = json_prompt | llmModel | json_parser

In [21]:
json_parser.get_format_instructions()

'Return a JSON object.'

In [16]:
res = json_chain.invoke({"question" : "Which is the biggest country?"})

In [17]:
type(res)

dict

#### Optionally, you can use Pydantic to define a custom output format

In [19]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
# from langchain_core.pydantic_v1 import BaseModel, Field # deprecated
from pydantic import BaseModel, Field

In [20]:
# Define a Pydantic Object with the desired output format.
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

In [22]:
# Define the parser referring the Pydantic Object
parser = JsonOutputParser(pydantic_object=Joke)

# Add the parser format instructions in the prompt definition.
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# Create a chain with the prompt and the parser
chain = prompt | chatModel | parser

chain.invoke({"query": "Tell me a joke."})

{'setup': 'Why did the scarecrow win an award?',
 'punchline': 'Because he was outstanding in his field.'}